<a href="https://colab.research.google.com/github/Pranshu1993/Transfer_Market_Values_Prediction/blob/master/notebooks/Copy_of_Deep_Learning_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Hyper Tuning**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
import pandas as pd

# Load the data from a CSV file
data = pd.read_csv('/content/Feature_selected.csv')

# Split the data into training and test sets
train_data = data.sample(frac=0.8, random_state=0)
test_data = data.drop(train_data.index)

# Separate the target variable from the features
X_train = train_data.drop(['Market Values'], axis=1)
y_train = train_data['Market Values']
X_test = test_data.drop(['Market Values'], axis=1)
y_test = test_data['Market Values']

# Define the model architecture
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32),
                               activation=hp.Choice('activation_' + str(i), values=['relu', 'tanh'])))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='mse',
        metrics=['mse'])
    return model

# Define the hyperparameter search space
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=3,
    directory='my_dir',
    project_name='helloworld')

# Perform the hyperparameter search
tuner.search(X_train, y_train,
             epochs=50,
             validation_data=(X_test, y_test))

# Print the best hyperparameters
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters: {best_hp}")

# Trial 10 Complete [00h 09m 24s]
# val_loss: 334365116596224.0

# Best val_loss So Far: 332913406465365.3
# Total elapsed time: 00h 51m 41s

Best hyperparameters: <keras_tuner.engine.hyperparameters.hyperparameters.HyperParameters object at 0x7f220cad76d0>


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load the dataset
df = pd.read_csv("/content/Feature_selected.csv")

# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)

# Define the dimensions of the noise input vector
noise_dim = 100

# Build the generator model
generator = Sequential()
generator.add(Dense(256, input_dim=noise_dim, activation='relu'))
generator.add(Dense(512, activation='relu'))
generator.add(Dense(1024, activation='relu'))
generator.add(Dense(df.shape[1], activation='linear'))
generator.add(Reshape((df.shape[1], 1)))

# Build the discriminator model
discriminator = Sequential()
discriminator.add(Flatten(input_shape=(df.shape[1], 1)))
discriminator.add(Dense(512, activation='relu'))
discriminator.add(Dense(256, activation='relu'))
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

# Build the GAN model
discriminator.trainable = False
gan = Sequential()
gan.add(generator)
gan.add(discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

# Train the GAN model
epochs = 100
batch_size = train_data.shape[0] # Set batch size equal to number of samples in train_data
for epoch in range(epochs):
    # Generate random noise as input to the generator
    noise = np.random.normal(0, 1, size=(batch_size, noise_dim))

    # Generate fake samples using the generator
    fake_data = generator.predict(noise)
    train_data_3D = train_data.values.reshape(train_data.shape[0], train_data.shape[1], 1)
    
    fake_data_3D = fake_data.reshape(fake_data.shape[0], fake_data.shape[1], 1)
    data_combined = np.concatenate((train_data_3D, fake_data_3D))

    # Labels for fake and real samples
    labels_combined = np.concatenate((np.zeros((batch_size, 1)), np.ones((batch_size, 1))))

    # Train the discriminator
    discriminator.trainable = True
    discriminator.train_on_batch(data_combined, labels_combined)

    # Train the generator
    noise = np.random.normal(0, 1, size=(batch_size, noise_dim))
    labels_mislabeled = np.ones((batch_size, 1))
    discriminator.trainable = False
    gan.train_on_batch(noise, labels_mislabeled)

    
    


# Generate predictions using the generator
# Generate predictions using the generator
noise = np.random.normal(0, 1, size=(test_data.shape[0], noise_dim))
predictions = generator.predict(noise)
predictions = predictions.reshape(test_data.shape[0], -1) * (df["Market Values"].max() - df["Market Values"].min()) + df["Market Values"].min()

# Calculate mean squared error
mse = mean_squared_error(test_data, predictions)
print("MSE:", mse)

gan.save("my_gan_model.h5")



37/37 [==============================] - 0s 4ms/step
MSE: 2.7264811127482426e+17


In [ ]:
# Importing necessary libraries
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

df=pd.read_csv("/content/Feature_selected.csv")
# Loading the dataset
X =df.drop(["Market Values"],axis=1)
y =df["Market Values"]   
 
# Splitting
train_X, test_X, train_y, test_y = train_test_split(X, y,
                      test_size = 0.3, random_state = 123)



Random Forest

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
df=pd.read_csv("/content/Feature_select.csv")
# Loading the dataset
X =df.drop(["Market Value"],axis=1)
y =df["Market Value"]   
 
# Splitting
train_X, test_X, train_y, test_y = train_test_split(X, y,
                      test_size = 0.3, random_state = 123)
import numpy as np
# Creating an instance of the Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=1000, random_state=42)

# Training the model
rf_regressor.fit(train_X, train_y)

# Making predictions on the test set
y_pred = rf_regressor.predict(test_X)

# Evaluating the model performance
mse = np.sqrt(mean_squared_error(test_y, y_pred))
print('root Mean Squared Error:', mse)

root Mean Squared Error: 11130243.572657559


**Feed Forward Neural Network**

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import pandas as pd

df=pd.read_csv("/content/Feature_selected.csv")
# Loading the dataset
X =df.drop(["Market Values"],axis=1)
y =df["Market Values"]   
# Splitting
train_X, test_X, train_y, test_y = train_test_split(X, y,
                      test_size = 0.3, random_state = 123)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


# Load the dataset


# Define the model architecture
model = Sequential()
model.add(Dense(512, input_dim=X.shape[1], activation='LeakyReLU')) # Input layer with 10 neurons
model.add(Dense(256, activation='LeakyReLU'))
model.add(Dense(128, activation='LeakyReLU')) # Hidden layer with 5 neurons
model.add(Dense(64, activation='LeakyReLU'))

model.add(Dense(1, activation='linear')) # Output layer with 1 neuron

# define callbacks to monitor the training process
early_stop = EarlyStopping(monitor='val_loss', patience=5)
checkpoint = ModelCheckpoint('model_weights.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1)


# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_X, train_y, epochs=100, batch_size=64, validation_split=0.2, callbacks=[early_stop, checkpoint])

model.save("feed_forward_NN.h5")



Epoch 1/100
30/34 [=========================>....] - ETA: 0s - loss: 468395107549184.0000
Epoch 1: val_loss improved from inf to 555376315465728.00000, saving model to model_weights.h5
34/34 [==============================] - 2s 16ms/step - loss: 458209894596608.0000 - val_loss: 555376315465728.0000
Epoch 2/100
28/34 [=======================>......] - ETA: 0s - loss: 440174186070016.0000
Epoch 2: val_loss improved from 555376315465728.00000 to 406125832830976.00000, saving model to model_weights.h5
34/34 [==============================] - 0s 11ms/step - loss: 407128976130048.0000 - val_loss: 406125832830976.0000
Epoch 3/100
31/34 [==========================>...] - ETA: 0s - loss: 311000964268032.0000
Epoch 3: val_loss improved from 406125832830976.00000 to 376631352885248.00000, saving model to model_weights.h5
34/34 [==============================] - 0s 10ms/step - loss: 305063541080064.0000 - val_loss: 376631352885248.0000
Epoch 4/100
32/34 [===========================>..] - ETA: 0s 

In [ ]:
row = test_X.iloc[5]
row2 = test_y.iloc[5]
# make prediction for the selected row
prediction = model.predict(np.array([row]))
print(row2)
# print the prediction
print(prediction)

1/1 [==============================] - 0s 24ms/step
8000000.0
[[7014145.]]


In [ ]:
df.head()

,minutes_played_overall,minutes_played_home,minutes_played_away,appearances_overall,appearances_home,appearances_away,goals_overall,goals_home,goals_away,assists_overall,...,Aggression,rank_in_club_top_scorer,Year,GK Positioning,penalty_goals,GK Handling,Defense Awareness/Marking,min_per_match,Standing Tackle,Market Values
0,-0.060716,-0.003595,-0.117620,-0.279406,-0.196729,-0.356941,-0.257054,-0.028932,-0.480840,-0.608484,...,1.316970,-0.308979,2017,-0.087152,-0.21616,-0.520990,1.037086,0.892382,1.077421,5000000.0
1,-0.041368,-0.145131,0.064552,-0.200715,-0.196729,-0.200484,-0.534581,-0.510339,-0.480840,-0.608484,...,1.316970,0.386336,2013,-0.087152,-0.21616,-0.520990,1.037086,0.761109,1.077421,4500000.0
2,-1.233552,-1.210274,-1.236417,-1.381076,-1.281678,-1.452141,-0.534581,-0.510339,-0.480840,-0.608484,...,-1.783763,1.197537,2022,-0.246409,-0.21616,-0.574494,-0.863667,-1.765896,-0.906437,3000000.0
3,-1.237237,-1.217532,-1.236417,-1.381076,-1.436670,-1.295684,-0.534581,-0.510339,-0.480840,-0.608484,...,-1.783763,-1.467838,2021,-0.246409,-0.21616,-0.574494,-0.863667,-1.897169,-0.906437,500000.0
4,0.993270,0.816583,1.155747,0.979646,0.578234,1.364086,0.853053,0.933884,0.629474,-0.154288,...,0.978708,-1.004294,2015,-0.564921,-0.21616,-0.574494,0.821092,0.662654,1.161840,1500000.0


In [ ]:
row =df.iloc[5]
row = row.drop('Market Values')

In [ ]:
# make prediction for the selected row
prediction = model.predict(np.array([row]))

# print the prediction
print(prediction)

1/1 [==============================] - 0s 30ms/step
[[12453427.]]


In [ ]:
df.iloc[5]['Market Values']

3500000.0

**LSTM**

In [ ]:
df=pd.read_csv("/content/Feature_selected.csv")
# Loading the dataset
X =df.drop(["Market Values"],axis=1)
y =df["Market Values"]   
 
# Splitting
train_X, test_X, train_y, test_y = train_test_split(X, y,
                      test_size = 0.3, random_state = 123)
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np




# # Convert DataFrame to NumPy array
# train_X = train_X.values

# # Reshape train_X to 3D shape
# train_X = train_X.reshape((train_X.shape[0], train_X.shape[1], 1))



# Reshape train_X to 3D shape
train_X = train_X.values.reshape((train_X.shape[0], train_X.shape[1], 1))




# define the model architecture
model = Sequential()
model.add(LSTM(units=64, input_shape=(train_X.shape[1], 1), return_sequences=True))

model.add(Dropout(0.2))
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=64))
model.add(Dropout(0.2))
model.add(Dense(units=1))

# compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# define callbacks to monitor the training process
early_stop = EarlyStopping(monitor='val_loss', patience=5)
checkpoint = ModelCheckpoint('model_weights.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1)
history = model.fit(train_X,train_y, epochs=10, batch_size=64, validation_split=0.2, callbacks=[early_stop, checkpoint])


model_json = model.to_json()
with open("model.json", "w") as json_file:
  json_file.write(model_json)


model.save_weights('final_model_weights.h5')

#evaluate the model on the test data

mse = model.evaluate(test_X, test_y)
print('Test MSE: %.3f' % mse)

Epoch 1/10
34/34 [==============================] - ETA: 0s - loss: 459675216642048.0000
Epoch 1: val_loss improved from inf to 563768480235520.00000, saving model to model_weights.h5
34/34 [==============================] - 13s 159ms/step - loss: 459675216642048.0000 - val_loss: 563768480235520.0000
Epoch 2/10
34/34 [==============================] - ETA: 0s - loss: 459675015315456.0000
Epoch 2: val_loss did not improve from 563768480235520.00000
34/34 [==============================] - 5s 150ms/step - loss: 459675015315456.0000 - val_loss: 563768480235520.0000
Epoch 3/10
34/34 [==============================] - ETA: 0s - loss: 459674948206592.0000
Epoch 3: val_loss improved from 563768480235520.00000 to 563768346017792.00000, saving model to model_weights.h5
34/34 [==============================] - 4s 117ms/step - loss: 459674948206592.0000 - val_loss: 563768346017792.0000
Epoch 4/10
34/34 [==============================] - ETA: 0s - loss: 459674948206592.0000
Epoch 4: val_loss did n

Auto Encoder

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df=pd.read_csv("/content/Feature_selected.csv")
# Loading the dataset
X =df.drop(["Market Values"],axis=1)
y =df["Market Values"]   
 
# Splitting
train_X, test_X, train_y, test_y = train_test_split(X, y,
                      test_size = 0.3, random_state = 123)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train_X)
X_test_scaled = scaler.transform(test_X)

# Define the autoencoder model
input_dim = X_train_scaled.shape[1]
encoding_dim = 10

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation='relu')(input_layer)
decoder = Dense(input_dim, activation='sigmoid')(encoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)

# Compile the model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Fit the model to the training data
autoencoder.fit(X_train_scaled, X_train_scaled,
                epochs=100,
                batch_size=64,
                shuffle=True,
                validation_data=(X_test_scaled, X_test_scaled))

# Extract the encoder part of the autoencoder as the feature extractor
encoder = Model(inputs=input_layer, outputs=encoder)
encoded_X_train = encoder.predict(X_train_scaled)
encoded_X_test = encoder.predict(X_test_scaled)

# Add a dense layer and an output layer to the encoder
model = Sequential()
model.add(Dense(32, input_dim=encoding_dim, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Fit the model to the encoded training data
model.fit(encoded_X_train, train_y,
          epochs=100,
          batch_size=64,
          shuffle=True,
          validation_data=(encoded_X_test,test_y))

# Make predictions using the test data
predictions = model.predict(encoded_X_test)




Epoch 1/100
43/43 [==============================] - 2s 14ms/step - loss: 1.2262 - val_loss: 1.2061
Epoch 2/100
43/43 [==============================] - 0s 7ms/step - loss: 1.1295 - val_loss: 1.0882
Epoch 3/100
43/43 [==============================] - 0s 6ms/step - loss: 1.0020 - val_loss: 0.9547
Epoch 4/100
43/43 [==============================] - 0s 10ms/step - loss: 0.8845 - val_loss: 0.8557
Epoch 5/100
43/43 [==============================] - 0s 7ms/step - loss: 0.8065 - val_loss: 0.7954
Epoch 6/100
43/43 [==============================] - 0s 5ms/step - loss: 0.7587 - val_loss: 0.7587
Epoch 7/100
43/43 [==============================] - 0s 7ms/step - loss: 0.7287 - val_loss: 0.7350
Epoch 8/100
43/43 [==============================] - 0s 8ms/step - loss: 0.7094 - val_loss: 0.7195
Epoch 9/100
43/43 [==============================] - 1s 15ms/step - loss: 0.6966 - val_loss: 0.7091
Epoch 10/100
43/43 [==============================] - 0s 6ms/step - loss: 0.6877 - val_loss: 0.7015
Epoch 

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load the dataset
df = pd.read_csv("/content/Feature_selected.csv")

# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)

# Define the dimensions of the noise input vector
noise_dim = 100

# Build the generator model
generator = Sequential()
generator.add(Dense(256, input_dim=noise_dim, activation='relu'))
generator.add(Dense(512, activation='relu'))
generator.add(Dense(1024, activation='relu'))
generator.add(Dense(df.shape[1], activation='linear'))
generator.add(Reshape((df.shape[1], 1)))

# Build the discriminator model
discriminator = Sequential()
discriminator.add(Flatten(input_shape=(df.shape[1], 1)))
discriminator.add(Dense(512, activation='relu'))
discriminator.add(Dense(256, activation='relu'))
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

# Build the GAN model
discriminator.trainable = False
gan = Sequential()
gan.add(generator)
gan.add(discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

# Train the GAN model
epochs = 100
batch_size = train_data.shape[0] # Set batch size equal to number of samples in train_data
for epoch in range(epochs):
    # Generate random noise as input to the generator
    noise = np.random.normal(0, 1, size=(batch_size, noise_dim))

    # Generate fake samples using the generator
    fake_data = generator.predict(noise)
    train_data_3D = train_data.values.reshape(train_data.shape[0], train_data.shape[1], 1)
    
    fake_data_3D = fake_data.reshape(fake_data.shape[0], fake_data.shape[1], 1)
    data_combined = np.concatenate((train_data_3D, fake_data_3D))

    # Labels for fake and real samples
    labels_combined = np.concatenate((np.zeros((batch_size, 1)), np.ones((batch_size, 1))))

    # Train the discriminator
    discriminator.trainable = True
    discriminator.train_on_batch(data_combined, labels_combined)

    # Train the generator
    noise = np.random.normal(0, 1, size=(batch_size, noise_dim))
    labels_mislabeled = np.ones((batch_size, 1))
    discriminator.trainable = False
    gan.train_on_batch(noise, labels_mislabeled)

    
    


# Generate predictions using the generator
# Generate predictions using the generator
noise = np.random.normal(0, 1, size=(test_data.shape[0], noise_dim))
predictions = generator.predict(noise)
predictions = predictions.reshape(test_data.shape[0], -1) * (df["Market Values"].max() - df["Market Values"].min()) + df["Market Values"].min()

# Calculate mean squared error
mse = mean_squared_error(test_data, predictions)
print("MSE:", mse)



37/37 [==============================] - 0s 4ms/step
MSE: 2.3474319155216214e+17


<bound method Series.count of 0       False
1       False
2       False
3       False
4       False
        ...  
3872    False
3873    False
3874    False
3875    False
3876    False
Name: Market Value, Length: 3877, dtype: bool>